In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =======

# ETL with NVTabular

In this notebook we are going to generate synthetic data and then create session-based features with [NVTabular](https://github.com/NVIDIA/NVTabular) to train a session-based recommendation model for next item prediction task.

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems. It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

- Import required libraries

In [2]:
import os
import glob

import torch 
import numpy as np

import cudf
import cupy as cp
import nvtabular as nvt

- Define Input/Output Path

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data/")

## Create a Synthetic Input Data

In [4]:
NUM_ROWS = 100000
inputs = {
    'session_id': np.random.randint(70000, 80000, NUM_ROWS),
    'day': np.random.randint(1, 10, NUM_ROWS),
    'item_id': np.random.lognormal(3., 1., NUM_ROWS).astype(np.int32),
    'category': np.random.randint(0, 332, NUM_ROWS),
    'timestamp/age_days': np.random.uniform(0, 1, NUM_ROWS),
    'timestamp/weekday/sin' : np.random.uniform(0, 1, NUM_ROWS),
    }
df = cudf.DataFrame(inputs)
df.head()

,session_id,day,item_id,category,timestamp/age_days,timestamp/weekday/sin
0,78349,5,5,63,0.895495,0.163206
1,76609,4,17,176,0.691784,0.227206
2,74052,3,22,150,0.360527,0.165966
3,76930,8,13,36,0.850611,0.595348
4,76695,3,26,278,0.294581,0.494827


## Feature Engineering with NVTabular

Deep Learning models require the input feature in a specific format. Categorical features needs to be continuous integers (0, ..., |C|) to use them with an embedding layer. We will use NVTabular to preprocess the categorical features, so that all categorical columns will be encoded to continuous integers. Note that we also add `1` after we categorify the categorical columns, the reason for that we want the encoded null values to start from `1` instead of `0` because we reserve `0` for padding the seqeunce features.

Here our goal is to create session-based features.  In this cell, we are creating temporal features and grouping them together at the session level, sorting the interactions by time. Note that we also trim each feature sequence in a  session to a certain length. Here, we use the NVTabular library so that we can easily preprocess and create features in a couple of lines on a GPU.

In [5]:
# Categorify categorical features

categ_feats = ['session_id', 'item_id', 'category'] >> nvt.ops.Categorify() >> nvt.ops.LambdaOp(lambda col: col +1)
# Define Groupby Workflow
groupby_feats = categ_feats + ['day', 'timestamp/age_days', 'timestamp/weekday/sin']

groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],     
        "day": ["first"],
        "timestamp/age_days": ["list"],
        'timestamp/weekday/sin': ["list"],
        },
    name_sep="-")

groupby_features_trim = (groupby_features['category-list', 'item_id-list', 'timestamp/age_days-list', 'timestamp/weekday/sin-list']) >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')

MINIMUM_SESSION_LENGTH = 2
selected_features = groupby_features['item_id-count', 'day-first', 'session_id'] + groupby_features_trim

filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)

workflow = nvt.Workflow(filtered_sessions)
dataset = nvt.Dataset(df, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

In [6]:
sessions_gdf.head(3)

,item_id-count,day-first,session_id,category-list_trim,item_id-list_trim,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim
0,24,8,2,"[281, 24, 322, 51, 65, 278, 92, 179, 80, 197, ...","[35, 8, 11, 11, 6, 30, 9, 24, 14, 15, 93, 15, ...","[0.892955009856083, 0.21738232902724808, 0.189...","[0.9274283040483893, 0.2950782656140717, 0.389..."
1,24,2,3,"[61, 301, 199, 92, 233, 25, 176, 268, 2, 120, ...","[246, 2, 33, 53, 27, 72, 7, 35, 38, 4, 12, 9, ...","[0.4695135267633095, 0.15847174044324397, 0.66...","[0.7007907651963626, 0.6100794479812083, 0.550..."
2,22,7,4,"[245, 251, 249, 19, 333, 46, 120, 94, 177, 77,...","[68, 54, 13, 6, 61, 13, 47, 6, 3, 36, 218, 88,...","[0.6086831914062396, 0.7630037586848936, 0.145...","[0.9836082578093043, 0.3248996724647304, 0.944..."


- we can save our workflow

In [7]:
workflow.save('workflow')

## Export pre-processed data by day

In [8]:
OUTPUT_FOLDER = os.environ.get("OUTPUT_FOLDER", INPUT_DATA_DIR + "sessions_by_day")
!mkdir -p $OUTPUT_FOLDER

In [9]:
# requires cudf + cupy + nvtabular + dask_cudf
from transformers4rec.utils.gpu_preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_FOLDER,
                       partition_col='day-first',
                       timestamp_col='session_id', 
                      )

Creating time-based splits: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 12.14it/s]


## Checking the pre-processing outputs¶

In [10]:
TRAIN_PATHS = sorted(glob.glob(os.path.join(OUTPUT_FOLDER, "1", "train.parquet")))

In [11]:
gdf = cudf.read_parquet(TRAIN_PATHS[0])
gdf.head()

,item_id-count,session_id,category-list_trim,item_id-list_trim,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim
0,19,50,"[105, 118, 101, 195, 301, 57, 19, 47, 239, 105...","[5, 5, 3, 25, 108, 8, 5, 12, 14, 13, 8, 5, 64,...","[0.996581398431122, 0.8647581530540058, 0.4271...","[0.4729119275072129, 0.5053625706372086, 0.538..."
1,19,54,"[77, 2, 154, 332, 14, 322, 292, 105, 128, 58, ...","[38, 19, 18, 17, 10, 118, 44, 25, 5, 35, 23, 1...","[0.3852559478929408, 0.9395631624840882, 0.548...","[0.1288564492663059, 0.08362282051577552, 0.99..."
2,19,61,"[331, 109, 202, 19, 319, 192, 322, 156, 118, 1...","[2, 108, 22, 22, 137, 130, 16, 34, 8, 15, 28, ...","[0.12819882765510937, 0.7805237237181775, 0.30...","[0.9059472783723667, 0.9850215005748464, 0.406..."
3,18,69,"[80, 59, 18, 233, 115, 47, 275, 159, 332, 298,...","[51, 25, 7, 50, 20, 57, 11, 6, 36, 4, 4, 54, 9...","[0.22780825515746927, 0.06990145868899689, 0.7...","[0.8841153650975748, 0.6324951113182498, 0.579..."
4,18,73,"[39, 323, 155, 314, 239, 295, 53, 133, 267, 85...","[19, 42, 3, 63, 10, 24, 23, 12, 19, 2, 9, 66, ...","[0.5612513114423124, 0.0924140824649482, 0.535...","[0.8043765626709053, 0.13703184234023247, 0.28..."
